# Basic directions

## Introduction to optimization and operations research

Michel Bierlaire


In [ ]:

import numpy as np
from teaching_optimization.linear_constraints import (
    draw_polyhedron_standard_form,
    StandardForm,
    LabeledDirection,
)


In this lab, you will explore **basic directions**: the directions obtained when a non-basic variable
is increased by one unit and the basic variables are adjusted to keep Ax = b satisfied (solving B d_B = −A_i).
You will learn how to build the full direction vector from its basic part, visualize it on simple polyhedra,
and decide whether a basic direction is **feasible** (i.e., keeps you inside the feasible region) or not.
This is the geometric move used by the **simplex method** to travel from one basic feasible solution
to a neighboring one; understanding these directions clarifies why simplex pivots work and when they fail
(e.g., infeasible or zero-length moves due to degeneracy). Work through the examples to connect the algebra
(basis matrix, indices, and linear solves) with the geometry (edges and vertices of the polyhedron).


We will implement the following functions:

- a function that calculates the basic part of the direction by solving the system $B d_B = -A_i$,
- a function that builds the direction in the full space,
- a function that calls all the previous ones to obtain a basic direction.

We will apply it to identify the vertices of the following polyhedrons.

## Polyhedron 1
$$P = \left\{
\begin{pmatrix}
x_1\\
x_2
\end{pmatrix} \in \mathbb{R}^2
| x_1+x_2  \geq 1,
x_1+x_2  \leq 2,
x_1  \geq 0,
x_2  \geq 0. \right\}.$$

## Polyhedron 2
$$ P = \left\{
\begin{pmatrix}
x_1\\
x_2
\end{pmatrix} \in \mathbb{R}^2
| x_1+x_2 \leq 1,
-x_1+2x_2 \leq 2, x_1 \geq 0, x_2 \geq 0 \right\}.$$

Prepare first polyhedron 1 so that we can use it for testing each function. It must first be written in standard form.

The polyhedron can be written in standard form by introducing non
negative slack variables, and rewriting the constraints in the form of equality constraints:

\begin{align*}
-x_1-x_2+x_3 & =-1,\\
x_1+x_2+x_4 &=2,\\
x_1,x_2,x_3,x_4 & \geq 0.
\end{align*}

The equality constraints in matrix form are
$$
Ax= b,
\text{ where }
A \in \mathbb{R}^{2\times 4},\; b \in \mathbb{R}^2,\; x \in \mathbb{R}^4,
$$
$$
A =
\begin{pmatrix*}[r]
-1&-1&1&0\\
1&1&0&1
\end{pmatrix*},\;
b =
\begin{pmatrix*}[r]
-1\\
2
\end{pmatrix*}.
$$

Provide the values of the matrix of the problem in standard form.

In [ ]:
standard_a = np.array([[-1, -1, 1, 0], [1, 1, 0, 1]])
print(standard_a)


In [ ]:
n_constraints, n_variables = standard_a.shape
print(f'{n_variables} variables, {n_constraints} constraints.')


Provide the values of the right-hand side of the problem in standard form.

In [ ]:
standard_b = np.array([-1, 2])
print(standard_b)


We select a feasible basic solution. We create the `StandardForm` object to have easy access to the feasible
basic solution.

In [ ]:
the_basic_indices = [1, 2]
the_standard_form = StandardForm(matrix=standard_a, vector=standard_b)
the_standard_form.basic_indices = the_basic_indices


Draw the polyhedron.

In [ ]:
draw_polyhedron_standard_form(matrix_a=standard_a, vector_b=standard_b)


## Function that calculates the basic part of the direction by solving the system $B d_B = -A_i$.
Complete it by replacing the ....

In [ ]:


def basic_part_basic_direction(
    constraint_matrix: np.ndarray, basic_indices: list[int], non_basic_index: int
) -> np.ndarray | None:
    """
    Function that calculates the basic part of the direction
    :param constraint_matrix: A numpy ndarray representing the constraint matrix.
    :param basic_indices: A list of integers representing the column indices to form the basis.
    :param non_basic_index: index of the non-basic variable associated with the basic direction.
    :return:
    """
    n_rows, n_columns = constraint_matrix.shape
    # Verify that the number of indices matches the number of rows in the matrix.
    if len(basic_indices) != n_rows:
        raise ValueError(
            f'The number of basic indices [{len(basic_indices)}] must match the number of '
            f'rows in the matrix [{n_rows}].'
        )

    # Verify each index is a valid column index of the matrix.
    max_index = n_columns - 1
    if not all(0 <= index <= max_index for index in basic_indices + [non_basic_index]):
        raise ValueError(
            'One or more indices are out of the valid column index range of the matrix.'
        )

    # Verify that the non-basic index is not in the basic list.
    if non_basic_index in basic_indices:
        raise ValueError(
            f'Non basic index {non_basic_index} also appears in the list of basic indices.'
        )

    # Extract the columns corresponding to the basic indices, in the provided order.
    basic_matrix = constraint_matrix[:, basic_indices]

    # Extract the column corresponding to the non-basic variable
    non_basic_column = constraint_matrix[
        :, non_basic_index
    ]

    # Solve the linear system and return the solution, if it exists.
    try:
        the_basic_direction = np.linalg.solve(basic_matrix, -non_basic_column)
        return the_basic_direction
    except np.linalg.LinAlgError:
        # The system does not have a solution
        return None



Test the function.  Expected result: $$\left(\begin{array}{c}-1 \\ 0  \end{array}\right).$$

In [ ]:
non_basic_index_1 = 0
basic_part_1 = basic_part_basic_direction(
    constraint_matrix=standard_a,
    basic_indices=the_basic_indices,
    non_basic_index=non_basic_index_1,
)
print(f'Basic part of d_{non_basic_index_1} = {basic_part_1}.')


Test again the function.  Expected result: $$\left(\begin{array}{c}-1 \\ -1  \end{array}\right).$$

In [ ]:
non_basic_index_2 = 3
basic_part_2 = basic_part_basic_direction(
    constraint_matrix=standard_a,
    basic_indices=the_basic_indices,
    non_basic_index=non_basic_index_2,
)
print(f'Basic part of d_{non_basic_index_2} = {basic_part_2}.')



## Function that builds the direction in the full space,
In a space of dimension $n$, we have a vector $d_B$ of dimension $m \leq n$, identified
by their indices. This function builds the vector in dimension $n$ where all non-basic entries are zero,
except one which is one.
Complete it by replacing the ....

In [ ]:
def build_basic_direction(
    number_of_variables: int,
    basic_part: np.ndarray,
    basic_indices: list[int],
    non_basic_index: int,
) -> np.ndarray:
    """
    Builds the vector in dimension n where all non-basic entries are zero, except one.

    :param number_of_variables: dimension n.
    :param basic_part: vector with the basic part of the direction.
    :param basic_indices: indices of basic variables in the space of dimension n.
    :param non_basic_index: index of the non-basic variable.
    :return: complete basic direction.
    """
    # Check that the number of basic variables is less or equal to n
    if len(basic_part) > number_of_variables:
        raise ValueError(
            f'The number of basic variables [{len(basic_part)}] must be less than or equal '
            f'to the dimension [{number_of_variables}].'
        )

    # Check that the two vectors have the same dimension
    if len(basic_part) != len(basic_indices):
        raise ValueError(
            f'The dimensions of the basic variables [{len(basic_part)}] and their '
            f'indices [{len(basic_indices)}] must match.'
        )

    # Check that all indices are valid
    if not all(
        0 <= index < number_of_variables for index in basic_indices + [non_basic_index]
    ):
        raise ValueError('All indices must be valid, that is between 0 and n-1.')

    # Initialize the complete solution vector with zeros
    complete_solution = np.zeros(number_of_variables)

    # Assign the values of basic variables to their corresponding positions
    for variable, index in zip(basic_part, basic_indices):
        complete_solution[index] = variable

    # Set the entry corresponding to the non-basic variable to 1.
    complete_solution[non_basic_index] = 1.0
    return complete_solution



Test the function.  Expected result: $$\left(\begin{array}{c}1 \\ -1 \\ 0\\ 0  \end{array}\right).$$

In [ ]:
full_direction_1 = build_basic_direction(
    number_of_variables=n_variables,
    basic_part=basic_part_1,
    basic_indices=the_basic_indices,
    non_basic_index=non_basic_index_1,
)
print(f'Basic direction d_{non_basic_index_1} = {full_direction_1}.')


We draw the polyhedron and the direction.

In [ ]:
draw_polyhedron_standard_form(
    matrix_a=standard_a,
    vector_b=standard_b,
    directions=[
        LabeledDirection(
            start=the_standard_form.basic_solution,
            direction=full_direction_1,
            width=0.05,
        )
    ],
)


Test again the function.  Expected result: $$\left(\begin{array}{c}0 \\ -1 \\ -1\\ 1  \end{array}\right).$$

In [ ]:
full_direction_2 = build_basic_direction(
    number_of_variables=n_variables,
    basic_part=basic_part_2,
    basic_indices=the_basic_indices,
    non_basic_index=non_basic_index_2,
)
print(f'Basic direction d_{non_basic_index_2} = {full_direction_2}.')


We draw the polyhedron and the direction.

In [ ]:

draw_polyhedron_standard_form(
    matrix_a=standard_a,
    vector_b=standard_b,
    directions=[
        LabeledDirection(
            start=the_standard_form.basic_solution,
            direction=full_direction_2,
            width=0.05,
        )
    ],
)



## Function that calls all the previous ones to obtain a basic direction.
Nothing needs to be done.The implementation is complete.

In [ ]:
def basic_direction(
    constraint_matrix: np.ndarray, basic_indices: list[int], non_basic_index: int
) -> np.ndarray | None:
    """

    :param constraint_matrix: A numpy ndarray representing the constraint matrix.
    :param basic_indices: A list of integers representing the column indices to form the basis.
    :param non_basic_index: index of the non-basic variable.
    :return: the basic direction.
    """
    n_constraints, n_variables = constraint_matrix.shape
    basic_part = basic_part_basic_direction(
        constraint_matrix=constraint_matrix,
        basic_indices=basic_indices,
        non_basic_index=non_basic_index,
    )
    if basic_part is None:
        print('Impossible to calculate the basic direction. Basic matrix is singular')
        return None

    full_direction = build_basic_direction(
        number_of_variables=n_variables,
        basic_part=basic_part,
        basic_indices=basic_indices,
        non_basic_index=non_basic_index,
    )
    return full_direction



We test it on the same polyhedron, with a different basis.

In [ ]:
the_basic_indices = [1, 3]
the_standard_form.basic_indices = the_basic_indices


We select the non basic variable.

In [ ]:
non_basic_index_1 = 0
full_direction_1 = basic_direction(
    constraint_matrix=standard_a,
    basic_indices=the_basic_indices,
    non_basic_index=non_basic_index_1,
)
print(full_direction_1)


We draw the polyhedron and the direction.

In [ ]:

draw_polyhedron_standard_form(
    matrix_a=standard_a,
    vector_b=standard_b,
    directions=[
        LabeledDirection(
            start=the_standard_form.basic_solution,
            direction=full_direction_1,
            width=0.05,
        )
    ],
)


We select another non basic variable.

In [ ]:
non_basic_index_2 = 2
full_direction_2 = basic_direction(
    constraint_matrix=standard_a,
    basic_indices=the_basic_indices,
    non_basic_index=non_basic_index_2,
)
print(full_direction_2)


We draw the polyhedron and the direction.

In [ ]:

draw_polyhedron_standard_form(
    matrix_a=standard_a,
    vector_b=standard_b,
    directions=[
        LabeledDirection(
            start=the_standard_form.basic_solution,
            direction=full_direction_2,
            width=0.05,
        )
    ],
)


## Polyhedron 2
We now investigate the second polyhedron.
$$ P = \left\{
\begin{pmatrix}
x_1\\
x_2
\end{pmatrix} \in \mathbb{R}^2
| x_1+x_2 \leq 1,
-x_1+2x_2 \leq 2, x_1 \geq 0, x_2 \geq 0 \right\}.$$

Data in standard form.

In [ ]:
standard_a = np.array([[1, 1, 1, 0], [-1, 2, 0, 1]])
standard_b = np.array([1, 2])


In [ ]:
n_constraints, n_variables = standard_a.shape
print(f'{n_variables} variables, {n_constraints} constraints.')


Draw the polyhedron.

In [ ]:
draw_polyhedron_standard_form(matrix_a=standard_a, vector_b=standard_b)


We select a basis.

In [ ]:
the_basic_indices = [0, 1]
the_standard_form = StandardForm(matrix=standard_a, vector=standard_b)
the_standard_form.basic_indices = the_basic_indices


Calculate the basic direction corresponding to non basic variable 2.

In [ ]:
non_basic_index_2 = 2
basic_direction_2 = basic_direction(
    constraint_matrix=standard_a,
    basic_indices=the_basic_indices,
    non_basic_index=non_basic_index_2,
)


In [ ]:
print(f'Feasible basic solution: {the_standard_form.basic_solution}')
print(f'Basic direction:         {basic_direction_2}')


If we move along the direction with a step $\alpha$, we obtain the point

In [ ]:
new_point = [
    f'{x:.3g} + {d:.3g} alpha'
    for x, d in zip(the_standard_form.basic_solution, basic_direction_2)
]
print(f'New point = [{", ".join(new_point)}]')


Note that, for any positive value of $\alpha$, the first coordinate is negative, and the point is infeasible.
Therefore, the basic direction is infeasible.
If we draw the polyhedron. We observe that the basic direction is indeed infeasible. It may happen
if the basic solution is degenerate, like in this example.

In [ ]:
draw_polyhedron_standard_form(
    matrix_a=standard_a,
    vector_b=standard_b,
    directions=[
        LabeledDirection(
            start=the_standard_form.basic_solution,
            direction=basic_direction_2,
            width=0.05,
        )
    ],
)


Calculate the basic direction corresponding to non basic variable 3.

In [ ]:
non_basic_index_3 = 3
basic_direction_3 = basic_direction(
    constraint_matrix=standard_a,
    basic_indices=the_basic_indices,
    non_basic_index=non_basic_index_3,
)


In [ ]:
print(f'Feasible basic solution: {the_standard_form.basic_solution}')
print(f'Basic direction:         {basic_direction_3}')


If we move along the direction with a step $\alpha$, we obtain the point

In [ ]:
new_point = [
    f'{x:.3g} + {d:.3g} alpha'
    for x, d in zip(the_standard_form.basic_solution, basic_direction_3)
]
print(f'New point = [{", ".join(new_point)}]')


Draw the polyhedron. We observe that this basic direction is feasible.

In [ ]:
draw_polyhedron_standard_form(
    matrix_a=standard_a,
    vector_b=standard_b,
    directions=[
        LabeledDirection(
            start=the_standard_form.basic_solution,
            direction=basic_direction_3,
            width=0.05,
        )
    ],
)


# Future usage.
These functions are implemented in the package, so that they can be reused in the future. Here is the syntax.

First, we create an object using the data:

In [ ]:
the_standard_form = StandardForm(matrix=standard_a, vector=standard_b)


Then, we set the basic indices.

In [ ]:
the_basic_indices = [0, 1]
the_standard_form.basic_indices = the_basic_indices


The basic solution is available as:

In [ ]:
basic_solution = the_standard_form.basic_solution
print(basic_solution)


And the basic directions can be obtained as follows:

In [ ]:
basic_direction_2 = the_standard_form.basic_direction(non_basic_index=2)
print(basic_direction_2)


In [ ]:
basic_direction_3 = the_standard_form.basic_direction(non_basic_index=3)
print(basic_direction_3)